In [80]:
import pandas as pd 
import plotly.express as px
import numpy as np


co2_label = "CO2 emissions (metric tons per capita)"
gdp_label = "gdpPercap"

gapminder_df = pd.read_csv('./gapminder_clean.csv')
gapminder_df = gapminder_df.replace([np.inf, -np.inf], np.nan).dropna()

gapminder_df..sort_values(by='pearson_corr', ascending=False)
hea.sort_values(by='Year')

,Unnamed: 0,Country Name,Year,"Agriculture, value added (% of GDP)",CO2 emissions (metric tons per capita),Domestic credit provided by financial sector (% of GDP),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Exports of goods and services (% of GDP),"Fertility rate, total (births per woman)",GDP growth (annual %),Imports of goods and services (% of GDP),"Industry, value added (% of GDP)","Inflation, GDP deflator (annual %)","Life expectancy at birth, total (years)",Population density (people per sq. km of land area),"Services, etc., value added (% of GDP)",pop,continent,gdpPercap
22,22,Algeria,1972,8.420846,1.843624,55.748145,142.029428,261.465251,20.449562,7.597,27.423969,25.726868,48.127965,-4.606461,51.171805,6.456244,43.451191,14760787.0,Africa,4182.663766
23,23,Algeria,1977,8.470141,2.368082,58.312256,231.710422,398.656113,30.586567,7.234,5.258586,41.740317,55.230902,11.927099,54.578268,7.427420,36.298962,17152804.0,Africa,4910.416756
17,17,Albania,1997,32.700160,0.490365,54.045926,680.688922,384.594958,9.745654,2.423,-10.837856,34.189995,15.377651,12.088706,72.898098,114.900766,51.922189,3428038.0,Europe,3193.054604
18,18,Albania,2002,24.235933,1.229541,47.193912,1561.122382,645.080154,19.598874,1.975,4.231371,44.335366,19.596072,2.410590,75.154610,111.350730,56.167995,3508512.0,Europe,4604.211737
19,19,Albania,2007,19.874798,1.322335,62.076755,1213.124369,679.861765,28.084222,1.635,5.900000,54.788201,25.334355,3.576195,76.470293,108.394781,54.790848,3600523.0,Europe,5937.029526


In [85]:
gapminder_df_1962 = gapminder_df[gapminder_df['Year'] == 1962]

gapminder_df_1962.head()

,Unnamed: 0,Country Name,Year,"Agriculture, value added (% of GDP)",CO2 emissions (metric tons per capita),Domestic credit provided by financial sector (% of GDP),Electric power consumption (kWh per capita),Energy use (kg of oil equivalent per capita),Exports of goods and services (% of GDP),"Fertility rate, total (births per woman)",GDP growth (annual %),Imports of goods and services (% of GDP),"Industry, value added (% of GDP)","Inflation, GDP deflator (annual %)","Life expectancy at birth, total (years)",Population density (people per sq. km of land area),"Services, etc., value added (% of GDP)",pop,continent,gdpPercap
2403,2403,Turkey,1962,53.875476,0.748241,23.001245,111.760694,410.021797,5.603985,6.175,5.571429,7.970112,17.407878,5.349372,46.836707,37.563485,28.716645,29788695.0,Europe,2322.869908


In [132]:
from scipy.stats import pearsonr

pearson_corr, p_value = pearsonr(gapminder_df[co2_label].values, gapminder_df[gdp_label].values)


print('The Correlation between GDP and CO2 is: \n')
print('Pearsons Correlation: ' + str(pearson_corr))
print('p value: ' + str(p_value))

The Correlation between GDP and CO2 is: 

Pearsons Correlation: 0.7505275641630997
p value: 1.9853468576120295e-95


In [131]:
# Find the year with the highest correlation 

def group_results(group):
    if len(group[co2_label].values) < 2: 
        return pd.Series({'pvalue': None, 'pearson_corr': None})
    
    pearson_corr, p_value = pearsonr(group[co2_label], group[gdp_label])
    return pd.Series({'pvalue': p_value, 'pearson_corr': pearson_corr })

grouped_results = gapminder_df.groupby('Year').apply(group_results).reset_index().sort_values(by='pearson_corr', ascending=False)

grouped_results

,Year,pvalue,pearson_corr
1,1967,1.000000e+00,1.000000
2,1972,1.544347e-16,0.936124
3,1977,7.359982e-16,0.898167
7,1997,2.105629e-22,0.818436
4,1982,9.790784e-13,0.815387
8,2002,4.459831e-20,0.783364
5,1987,2.999945e-10,0.747282
6,1992,5.028110e-13,0.744709
9,2007,1.051016e-12,0.649742
0,1962,NaN,NaN


While the pearson correlation for 1967 is 1, the p-value is also 1 and is not statistically significant. Instead 1972 is the year with the highest correlation between GDP and CO2 Emissions.

In [26]:
import plotly.express as px
highest_corr_df = gapminder_df[gapminder_df['Year']== highest_corr_year]
fig = px.scatter(highest_corr_df, x=co2_label, y=gdp_label, color="continent",
                 title="CO2 Emission Per Capita v/s GDP Per Capita", size="pop")

fig.show()

### Time for some questions!

### Q1 

What is the relationship between `continent` and `'Energy use (kg of oil equivalent per capita)'`? 

Well, the first step is to actually see the distribution of energy used per continent.

In [167]:
energy_by_continent = gapminder_df.groupby(['continent', 'Year']).apply(lambda x: pd.Series({'energy':sum(x['Energy use (kg of oil equivalent per capita)'])})).reset_index()

energy_by_continent

fig = px.box(energy_by_continent, x="continent", y="energy")

fig.show()

Looks like energy usage is the highest for Europe, and drastically falls off for Asia and Americas. I have a hunch this is because Europe is huge + has higher GDP per capita. Furthermore, since Europe technically had the Industrial revolution first, their total CO2 emissions have to be much higher.

### Q2

Is there a significant difference between Europe and North America with respect to 'Imports of goods and services (% of GDP)' in the years after 1990?

For simplicity, I will only be considering US + Canada + Mexico as North American countries. 


In [155]:
q2_df = gapminder_df[gapminder_df['Year'] > 1990]

north_american_countries = ['United States', 'Canada', 'Mexico']
goods_label = 'Imports of goods and services (% of GDP)'

na_df = q2_df[q2_df['Country Name'].isin(north_american_countries)]
eu_df = q2_df[q2_df['continent']=='Europe']

eu_goods = eu_df.groupby('Year').apply(lambda x: pd.Series({'EU Import': np.sum(x[goods_label])})).reset_index()
na_goods = na_df.groupby('Year').apply(lambda x: pd.Series({'NA Import': np.sum(x[goods_label])})).reset_index()

merged_goods = eu_goods.merge(na_goods, on='Year')

px.bar(merged_goods, x='Year', y=['EU Import', 'NA Import'], barmode='group')

Europe had much higher imports than NA in almost every year after 1990. 

### Q3

What is the country that has the highest 'Population density (people per sq. km of land area)' across all years? (i.e., which country has the highest average ranking in this category across each time point in the dataset?)

In [35]:
from collections import defaultdict
import operator

pd_label = 'Population density (people per sq. km of land area)'

pd_df = gapminder_df[['Country Name', pd_label, 'Year']]

rankings = defaultdict(list)

for year in pd_df['Year'].unique():
    year_df = pd_df[pd_df['Year'] == year].sort_values(by=pd_label, ascending=False)
    for index,row in year_df.iterrows():
        rankings[row['Country Name']].append(index)


rankings = { key:np.mean(rankings[key]) for key in rankings }

max(rankings.items(), key=operator.itemgetter(1))[0]


'Zimbabwe'

Zimbabwe had the highest average ranking across all years in Population Density.

### Q4

What country has shown the greatest increase in 'Life expectancy at birth, total (years)' since 1962?


In [79]:
le_label = 'Life expectancy at birth, total (years)'

le_df = gapminder_df[['Year', 'Country Name', le_label]]

max_year = max(le_df['Year'].unique())

# Get first and last available values for each country 
le_df = le_df.sort_values(by="Year").groupby(['Country Name']).agg(['first', 'last'])

# Filter to only include results after 1962
le_df = le_df[le_df['Year']['first'] >= 1962]

le_df['le_diff'] = le_df[le_label]['last'] - le_df[le_label]['first']

le_df = le_df.reset_index()
px.scatter(le_df, y='le_diff', color='Country Name')

Turkey's life expectancy has increased by 26 since first recorded after 1962!